## My First Advent of Code !!

### Day 1

In [1]:
# WARNING : this can take a while to run, depending on repeat length

def day_one(repeat):

    count=0
    count_list=[]
    i=0

    print("start")

    for _ in range(repeat):
        with open("./input.txt",'r') as filein:
            for line in filein:
                if line[0] == "-":
                    count= count - float(line[1:])
                    if count in count_list:
                        print ("repeat found for "+str(count))
                        return
                    else:
                        count_list.append(count)

                elif line[0] == "+":
                    count = count + float(line[1:])
                    if count in count_list:
                        print ("repeat found for "+str(count))  
                        return
                    else:
                        count_list.append(count)

                else:
                    print ("urgh, something wrong")
                    
    print (count)
    print ("end")



In [2]:
day_one(1)   # 466
day_one(150)   # 750

start
466.0
end
start
repeat found for 750.0


### Day 2

In [3]:
def day_two():
    with open("input (1).txt","r") as filein:
        two_count = 0
        three_count = 0
        for line in filein:
            letter_list = (list(line)[:-1])
            for letter in letter_list: 
                if letter_list.count(letter) == 2:
                    two_count += 1
                    break
                else:
                    continue
            for letter in letter_list:                        
                if letter_list.count(letter) == 3:
                    three_count += 1
                    break
                else:
                    continue
                
    checksum = two_count * three_count
    print (checksum)
    
    ID_list = []
    with open("input (1).txt","r") as filein:
        for line in filein:
            ID_list.append(line[:-1])
            
    for ID1_index,ID1 in enumerate(ID_list):
        for ID2 in ID_list:
            # this is the juice. Go through removing one letter at a time, from both arrays, and see if they match.
            for letter_index in range(len(list((ID1)))):
                if (list(ID1)[:letter_index]+list(ID1)[letter_index+1:]) == (list(ID2)[:letter_index]+list(ID2)[letter_index+1:]):
                    if ID1 != ID2:
                        print ("match found:"+ID1+"/"+ID2)
                        return

                else:
                    continue
                
            
    

In [4]:
day_two() # 5166
          # cypueihajytordkgzxfqplbwn

5166
match found:cypueihajytotrdkgzxfqplbwn/cypueihajytomrdkgzxfqplbwn
match found:cypueihajytomrdkgzxfqplbwn/cypueihajytotrdkgzxfqplbwn


### Day 3

In [5]:
import numpy as np
import re

def day_three():
    fabric = np.zeros([1000,1000])
    with open("input (2).txt","r") as filein:
        for line in filein:
            ID = line.split()[0]
            left=int(re.search(r'(\d*)\,(\d*)',line.split()[2]).group(1))
            top=int(re.search(r'(\d*)\,(\d*)',line.split()[2]).group(2))
            width=int(re.search(r'(\d*)x(\d*)',line.split()[3]).group(1))
            height=int(re.search(r'(\d*)x(\d*)',line.split()[3]).group(2))
            for i in range(height):
                for j in range(width):
                    fabric[top+i][left+j]+=1
    # np.where is suuupppeerr useful here             
    overlap=fabric[np.where(fabric>1)]
    print (len(overlap))
    
## part two
    a=np.where(fabric==1)
    fabric_indices=[]
    # list of array indices where equal to 1
    for i,j in zip(a[0],a[1]):
        fabric_indices.append([i,j])
    
    with open("input (2).txt","r") as filein:                
        for line in filein:
            fabric_individual = np.zeros([1000,1000])
            ID = line.split()[0]
            left=int(re.search(r'(\d*)\,(\d*)',line.split()[2]).group(1))
            top=int(re.search(r'(\d*)\,(\d*)',line.split()[2]).group(2))
            width=int(re.search(r'(\d*)x(\d*)',line.split()[3]).group(1))
            height=int(re.search(r'(\d*)x(\d*)',line.split()[3]).group(2))
            for i in range(height):
                for j in range(width):
                    fabric_individual[top+i][left+j]=1
                    
            a=np.where(fabric_individual==1)
            individual_indices=[]
            # list of array indices for particular fabric cut where equal to 1
            for i,j in zip(a[0],a[1]):
                individual_indices.append([i,j])
            
            # if all of the array indices are also 1 in the fabric_array, then we have our match
            if all(elem in fabric_indices for elem in individual_indices):
                print (ID)
                return
    
                
    
# for sanity checks..
#    with open("fabric.txt","w") as fileout:
#        with np.printoptions(threshold=np.inf):
#            fileout.write(str(overlap))

In [ ]:
day_three() #118322
            # 1178

### Day 4

In [7]:
import pandas as pd

# I've probably went a bit overkill here, but I went from clarity over brevity.

def day_four():

    # read in data and sort by datetime stamp       
    df=pd.read_csv("input (3).txt",delimiter="]",header=None,names=["time","event"])
    df=df.sort_values(by='time')
    
    # create ID column row
    df['ID'] = pd.Series(np.random.randint(0,10,size=len(df['time'])))
                         
    # set ID column row
    for index, row in df.iterrows():
        if 'Guard' in row['event']:
            ID_number = int(row['event'].split()[1][1:])
            df.set_value(index, 'ID', ID_number)
        else:
            df.set_value(index, 'ID', ID_number)
    
    # iterate through guard ID_numbers
    guard_total=[]
    guard_break_down=[]
    for ID_number in df.ID.unique():
        # re-set index for each individual guard
        df_guard = df[df['ID'] == ID_number].reset_index(drop=True)   
        # set a tally for each minute between midnight and 1am
        sleep_count = np.zeros(60)      
        for index, row in df_guard.iterrows():
            if 'asleep' in row['event']:
                # check it's not the final row in the dataframe
                if (index+1 != len(df_guard)):
                    # add 1 to the minute tally between asleep and waking minutes
                    if 'wakes' in df_guard.iloc[index+1]['event']:
                        start=int(df_guard.iloc[index]['time'].split()[1][-2:])
                        end=int(df_guard.iloc[index+1]['time'].split()[1][-2:])
                        sleep_count[start:end]+=1
                    # if final event for the day, add 1 to the minute tally from minute asleep to 1am (though don't think this happens)
                    else:
                        start=int(df_guard.iloc[index]['time'].split()[1][-2:])
                        sleep_count[start:]+=1
                # if final row in dataframe, add 1 to the minute tally from minute asleep to 1am (though don't think this happens)
                else:
                    start=int(df_guard.iloc[index]['time'].split()[1][-2:])
                    sleep_count[start:]+=1

        guard_total.append([ID_number,sleep_count.sum()])
        guard_break_down.append([ID_number,sleep_count])
    return np.array(guard_total), guard_break_down

In [8]:
# I got sloppy here and done stuff by inspection, but this was the easiest way (for me)

guard_total, guard_break_down = day_four()
print (guard_total)
print (max(guard_total[:,1]))
# from inspection
# ID = 131
# minute = 36
# part 1: 4716

guard_max=[]
for break_down in guard_break_down:
    print (break_down[0],max(break_down[1]))
    
# from inspection
# ID = 2389
# minute = 49
# part 2: 117061

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


[[1487.  461.]
 [1493.  429.]
 [ 103.  291.]
 [2389.  427.]
 [3391.  366.]
 [3253.  229.]
 [ 641.  402.]
 [ 131.  489.]
 [2797.  349.]
 [2437.  358.]
 [2447.  448.]
 [ 619.  375.]
 [1613.  334.]
 [1033.  252.]
 [2969.  235.]
 [ 443.  371.]
 [2089.  330.]
 [1373.  367.]
 [2417.  432.]
 [3023.  150.]
 [2683.    0.]
 [1973.    0.]
 [1913.    0.]]
489.0
1487 13.0
1493 14.0
103 10.0
2389 15.0
3391 11.0
3253 9.0
641 14.0
131 14.0
2797 9.0
2437 12.0
2447 11.0
619 12.0
1613 11.0
1033 10.0
2969 7.0
443 13.0
2089 9.0
1373 11.0
2417 11.0
3023 5.0
2683 0.0
1973 0.0
1913 0.0


### Day 5

In [16]:
## at first I tried iterating through each element of the string and comparing letter to letters before and after.
## Using regex is much less clunky

import string
import re

def react_polymer(polymer_string):
    lowers=list(string.ascii_lowercase)
    proceed = True
    while proceed==True:
        proceed=False
        for letter in lowers:
            Ul = letter.upper()+letter
            if re.search(Ul,polymer_string):
                regex=re.search(r'(.*)'+Ul+'(.*)',polymer_string)
                polymer_string=regex.group(1)+regex.group(2)
                proceed=True
            lU = letter + letter.upper()
            if re.search(lU,polymer_string):
                regex=re.search(r'(.*)'+lU+'(.*)',polymer_string)
                polymer_string=regex.group(1)+regex.group(2)
                proceed=True

    return (len(polymer_string))



In [34]:
# part one
# polymer_string = "dabAcCaCBAcCcaDA"   # example string
polymer_string = open("input (4).txt","r").read()   # question string
units=react_polymer(polymer_string) 
print ("part one: "+str(units))# answer 11546

# part two
results=[]
lowers=list(string.ascii_lowercase)
for letter in lowers:
    polymer_string_reduced = re.sub(letter,'',polymer_string)
    polymer_string_reduced = re.sub(letter.upper(),'',polymer_string_reduced)
    results.append([letter,react_polymer(polymer_string_reduced)])
# I need to do this a better way than appending to lists...making an array..transforming to ints...finding min...
print ("part two: "+str(min([int(x) for x in np.array(results)[:,1]]))) # answer 

part one: 11546
part two: 5124


### Day 6

In [34]:
import numpy as np

 # from puzzle example input
coordinates = np.array([
[1, 1],
[1, 6],
[8, 3],
[3, 4],
[5, 5],
[8, 9]])  

# for question
coordinates=[]
with open("input (5).txt","r") as filein:
    for line in filein:
        coordinates.append([int(x.split("\n")[0]) for x in line.split(",")])
        
coordinates = np.array(coordinates) 

minX = min(coordinates[:,0]) 
maxX = max(coordinates[:,0]) 
minY = min(coordinates[:,1]) 
maxY = max(coordinates[:,1]) 

coordinates[:,0] = [x-minX for x in coordinates[:,0]]
coordinates[:,1] = [y-minY for y in coordinates[:,1]]

height=maxY-minY+1
width=maxX-minX+1

charlist = [['---------']*width]*height
location_map = np.array(charlist)

safe_region_count=0
for i in range(location_map.shape[0]):
    for j in range(location_map.shape[1]):
        distances = [np.absolute(i-y)+np.absolute(j-x) for [x,y] in coordinates]
        if sum(distances) < 10000:
            safe_region_count +=1
        if len(np.where(distances==min(distances))[0]) > 1:
            location_map[i,j] = "........."
        else:      
            location_map[i,j] = str(coordinates[distances.index(min(distances))])

finite_locations_count=[]
for coordinate in coordinates:
    x=str(coordinate)
    if (x in location_map[0,:]) or (x in location_map[-1,:]) or (x in location_map[:,0]) or (x in location_map[:,-1]):
        pass
    else:
        finite_locations_count.append((location_map==x).sum())
        
print ("part one: "+str(max(finite_locations_count)))
print ("part two: "+str(safe_region_count))


part one: 4475
part two: 35237


### Day 7

This seems a good time to get my head around Python default dictionaries. Nice tutorial here: https://www.accelebrate.com/blog/using-defaultdict-python/

In [146]:
from collections import defaultdict

answer=[]

instruction_list = []
#with open("input (6).txt","r") as filein:
with open("Day 6 example","r") as filein:
    for line in filein:
        instruction_list.append([line.split()[1],line.split()[-3]])
        
steps = defaultdict(list)

for [prerequisite, step] in instruction_list:
    steps[step].append(prerequisite)
    
step_list = []
for row in instruction_list:
    step_list.append(row[1])

for row in instruction_list:
    prerequisite = row[0]
    if prerequisite not in step_list and prerequisite not in steps:
        steps[prerequisite]

### this is the bit that needs to loop until all empty
while all(value == [] for value in steps.values()) is False:
    steps_ready = {k: v for k, v in steps.items() if v==[]}
    next_step = min(steps_ready.keys()) 
    answer.append(next_step)
    # need to remove from steps list
    del steps[next_step]

    for key,value in steps.items():
        value = [item for item in value if item != next_step]
        steps[key] = value
        
    if len(steps) == 1:
        for key in steps:
            answer.append(key)

In [147]:
print(answer)  # ['I', 'O', 'F', 'S', 'J', 'Q', 'D', 'U', 'W', 'A', 'P', 'X', 'E', 'L', 'N', 'V', 'Y', 'Z', 'M', 'H', 'T', 'B', 'C', 'R', 'G', 'K']

['C', 'A', 'B', 'D', 'F', 'E']


In [8]:
from collections import defaultdict

time_diff = -4
num_elves = 1

answer=[]

instruction_list = []
with open("input (6).txt","r") as filein:
#with open("Day 6 example","r") as filein:
    for line in filein:
        instruction_list.append([line.split()[1],line.split()[-3]])
        
steps = defaultdict(list)
for [prerequisite, step] in instruction_list:
    steps[step].append(prerequisite)    
step_list = []
for row in instruction_list:
    step_list.append(row[1])
for row in instruction_list:
    prerequisite = row[0]
    if prerequisite not in step_list and prerequisite not in steps:
        steps[prerequisite]=0

elves = defaultdict(list)
for elf in range(num_elves):
    elves[elf].append(True)

def question_seven_part_two(steps,elves):
    second = 0
    
    ### this is the bit that needs to loop until all empty
    while all(value == [] for value in steps.values()) is False:
        print (steps,"\n",elves)
        steps_ready = {k: v for k, v in steps.items() if v==0}
        if len(steps_ready)>0:
            next_steps = steps_ready.keys()
            for step in next_steps:
                if elf_available(elves):
                    elves = allocate_task(elves,step)            
                    answer.append(step)
                    steps[step]=ord(step)+time_diff           
        second +=1
        elves = deallocate_tasks(elves)
        
        delete_key=[]
        for key, value in steps.items():
            if type(value)==int:
                if value > 0:
                    steps[key] = value-1                
            if value == 1:
                delete_key.append(key)
        
        for key in delete_key:
            del steps[key]
                
        for key,value in steps.items():
            if type(value)==list:
                value = [item for item in value if item not in delete_key]
                steps[key] = value
                
        for key,value in steps.items():
            if value == []:
                steps[key]=0
            
        if len(steps) == 1:
            for key,value in steps.items():
                answer.append(key)
                second += (ord(key)+time_diff)
                return answer,second

        print ("\n")
        
def elf_available(elves):
    for k,v in elves.items():
        if v[0] == True:
            return True
    return False

def allocate_task(elves,next_step):
    seconds = ord(next_step) + time_diff
    available_elves = {k: v for k, v in elves.items() if v[0]==True}
    elf = min(available_elves.keys())
    for second in range(seconds):
        elves[elf].insert(0,False)        
    return elves

def deallocate_tasks(elves):
    for k,v in elves.items():
        if v[0] == False:
            elves[k] = v[1:]
    return elves

In [9]:
question_seven_part_two(steps,elves) # 931

defaultdict(<class 'list'>, {'W': ['O'], 'V': ['S', 'Q', 'X', 'J'], 'B': ['Z', 'E', 'Y', 'H', 'T', 'S', 'Q', 'P', 'I', 'V'], 'R': ['F', 'U', 'V', 'E', 'W', 'C', 'O', 'M'], 'D': ['I', 'S', 'Q', 'J'], 'P': ['W'], 'E': ['J', 'X'], 'N': ['P', 'X', 'A', 'U'], 'K': ['D', 'C', 'R', 'G', 'Q', 'U', 'T', 'E', 'H', 'X', 'B'], 'H': ['L', 'Q', 'I', 'N', 'A', 'D', 'M', 'O'], 'T': ['A', 'M', 'L', 'N', 'Z', 'X', 'U'], 'C': ['N', 'T', 'B', 'L', 'V', 'J', 'Y', 'M', 'U', 'S'], 'Y': ['U', 'V', 'O', 'A', 'W', 'S', 'Q'], 'G': ['L', 'N', 'E', 'C', 'D', 'S', 'M', 'A', 'H', 'R'], 'M': ['Z', 'J', 'L', 'P', 'Q'], 'Q': ['J'], 'U': ['Q'], 'L': ['E', 'D'], 'S': ['O'], 'A': ['W'], 'J': ['S', 'I'], 'F': ['O'], 'O': 0, 'Z': 0, 'I': 0, 'X': 0}) 
 defaultdict(<class 'list'>, {0: [True], 1: [True], 2: [True], 3: [True], 4: [True]})


defaultdict(<class 'list'>, {'W': ['O'], 'V': ['S', 'Q', 'X', 'J'], 'B': ['Z', 'E', 'Y', 'H', 'T', 'S', 'Q', 'P', 'I', 'V'], 'R': ['F', 'U', 'V', 'E', 'W', 'C', 'O', 'M'], 'D': ['I', 'S', 'Q

defaultdict(<class 'list'>, {'B': ['Y', 'H', 'T'], 'R': ['C', 'M'], 'N': 55, 'K': ['C', 'R', 'G', 'T', 'H', 'B'], 'H': ['L', 'N', 'M'], 'T': ['M', 'L', 'N'], 'C': ['N', 'T', 'B', 'L', 'Y', 'M'], 'Y': 67, 'G': ['L', 'N', 'C', 'M', 'H', 'R'], 'M': ['L'], 'L': 36}) 
 defaultdict(<class 'list'>, {0: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True], 1: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False


defaultdict(<class 'list'>, {'R': 17, 'K': ['R', 'G'], 'G': ['R']}) 
 defaultdict(<class 'list'>, {0: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True], 1: [True], 2: [True], 3: [True], 4: [True]})


defaultdict(<class 'list'>, {'R': 16, 'K': ['R', 'G'], 'G': ['R']}) 
 defaultdict(<class 'list'>, {0: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True], 1: [True], 2: [True], 3: [True], 4: [True]})


defaultdict(<class 'list'>, {'R': 15, 'K': ['R', 'G'], 'G': ['R']}) 
 defaultdict(<class 'list'>, {0: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True], 1: [True], 2: [True], 3: [True], 4: [True]})


defaultdict(<class 'list'>, {'R': 14, 'K': ['R', 'G'], 'G': ['R']}) 
 defaultdict(<class 'list'>, {0: [False, False, False, False, False, False, False, False, False, False, False, False, False

(['O',
  'Z',
  'I',
  'X',
  'W',
  'S',
  'F',
  'J',
  'P',
  'A',
  'E',
  'Q',
  'V',
  'D',
  'U',
  'L',
  'N',
  'Y',
  'M',
  'H',
  'T',
  'B',
  'C',
  'R',
  'G',
  'K'],
 931)